In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.2 MB/s eta 0:00:00


In [ ]:
import zipfile
import os

# Specify the path of the zip file
zip_file_path = '/content/source.zip'

# Specify the destination directory where the folder will be extracted
extract_dir = '/content/'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Optionally, remove the zip file after extraction
os.remove(zip_file_path)

In [ ]:
import zipfile
import os

# Specify the path of the zip file
zip_file_path = '/content/webanno.zip'

# Specify the destination directory where the folder will be extracted
extract_dir = '/content/'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Optionally, remove the zip file after extraction
os.remove(zip_file_path)

In [ ]:
import os
import pandas as pd
import csv

# Initialize lists to store source text and annotation dataframes
source_dfs = []
annotation_dfs = []

# Walk through the source directory and read TXT files
source_dir = '/content/source'
annotation_dir = '/content/webanno'

for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.endswith('.txt'):
            try:
                with open(os.path.join(root, file), 'r', encoding='utf-8', errors='ignore') as f:
                    source_text = f.read()

                # Create a dataframe for the source text
                df_source = pd.DataFrame([source_text], columns=['Source_Text'])
                # Add a column with the name of the txt file
                df_source['File_Name'] = file

                # Append the dataframe to the list
                source_dfs.append(df_source)

                # Build the corresponding annotation directory path and TSV filename
                annotation_folder = file  # Use the complete file name, including '.txt', as folder name
                tsv_file = 'annotation.tsv'

                # Build the full path to the TSV file
                tsv_path = os.path.join(annotation_dir, annotation_folder, tsv_file)

                if os.path.exists(tsv_path):
                    # Read the TSV into a dataframe
                    df_annotation = pd.read_csv(tsv_path, sep='\t', comment='#', header=None,
                                                names=['Token_ID', 'Character_Offset', 'Token', 'Annotation_1', 'Annotation_2', 'Annotation_3', 'Annotation_4'],
                                                encoding='utf-8', na_filter=False, keep_default_na=False, on_bad_lines='skip',
                                                quoting=csv.QUOTE_NONE)  # Add this line
                    # Add a column with the name of the txt file
                    df_annotation['File_Name'] = file

                    # Append the dataframe to the list
                    annotation_dfs.append(df_annotation)

            except Exception as e:
                print(f"An error occurred with file {file}. Error: {str(e)}")

# Concatenate all dataframes
df_all_source = pd.concat(source_dfs, ignore_index=True)
df_all_annotation = pd.concat(annotation_dfs, ignore_index=True)

# Print the combined dataframes to check
print(df_all_source)
print(df_all_annotation)

                                          Source_Text  \
0   Besides these, the island of Java, and particu...   
1   Spanish Mackril are in shape and colour like o...   
2   \nFLORIDA WATER. — This perfume, which, of cou...   
3   He coughed slightly and said, ‘Yes,’ and the w...   
4   At the time of my examination, it had become f...   
..                                                ...   
80  Why? you may as reasonably expect to preserve ...   
81  320. Italian Snuffs \n \nPut into a mortar, or...   
82  \nSection1 .\n\n\nIn the first place, I am of ...   
83  It may be distinguished from them by its very ...   
84  \n\nIt seems highly probable that, as has been...   

                                            File_Name  
0   024E John Hawkesworth 1773 An Account__43_dist...  
1   023E William Dampier 1705 Voyages and Descript...  
2   065E Ernest J Parry 1925 Parry's Cyclopedia of...  
3   049E H V Morton 1926 H V Mortons London__122_d...  
4   006E W Murdoch 1858 Report of t

In [ ]:
df_all_annotation.Annotation_1.unique()

array(['_', 'Smell\\_Word[1]|Quality[2]', 'Smell\\_Source[3]', ...,
       'Perceiver[108]', 'Location[110]', 'Circumstances[111]'],
      dtype=object)

In [ ]:

def detokenize(df):
    sentences = []
    sentence = []

    for index, row in df.iterrows():
        token = row['Token']
        label = row['Annotation_1']

        # If there's a label associated with the token
        if pd.notnull(label) and label != '':
            sentence.append(token)

        # If there's no label, it means the end of a sentence
        else:
            if sentence:  # If the sentence is not empty
                sentences.append(' '.join(sentence))
            sentence = []  # Start a new sentence

    # Make sure to add the last sentence if the text does not end with an empty label
    if sentence:
        sentences.append(' '.join(sentence))

    return sentences

# Call the detokenize function
sentences = detokenize(df_all_annotation)
print(sentences)

['Besides these , the island of Java , and particularly the country round Batavia , produces many kinds of fruit which were not in season during our stay ; we were also told that apples , strawberries , and many other fruits from Europe , had been planted up in the mountains , and flourished there in great luxuriance . We saw several fruits preserved in sugar , that we did not see recent from the tree , one of which is called Kimkit , and another Boa Atap : and here are several others which are eaten only by the natives , particularly the Kellor , the Guilindina , the Moringa , and the Soccum . The Soccum is of the same kind with the bread-fruit in the South Sea islands , but so much inferior , that if it had not been for the similitude in the outward appearance both of the fruit and the tree , we should not have referred it to that class . These and some others do not merit to be particularly mentioned . The quantity of fruit that is consumed at Batavia is incredible ; but that which 

In [ ]:
# import os
# import pandas as pd

# # Initialize lists to store source text and annotation dataframes
# source_dfs = []
# annotation_dfs = []

# # Walk through the source directory and read TXT files
# source_dir = '/content/source'
# annotation_dir = '/content/webanno'

# for root, dirs, files in os.walk(source_dir):
#     for file in files:
#         if file.endswith('.txt'):
#             try:
#                 with open(os.path.join(root, file), 'r', encoding='utf-8', errors='ignore') as f:
#                     source_text = f.read()

#                 # Create a dataframe for the source text
#                 df_source = pd.DataFrame([source_text], columns=['Source_Text'])
#                 # Add a column with the name of the txt file
#                 df_source['File_Name'] = file

#                 # Append the dataframe to the list
#                 source_dfs.append(df_source)

#                 # Build the corresponding annotation directory path and TSV filename
#                 annotation_folder = file  # Use the complete file name, including '.txt', as folder name
#                 tsv_file = 'annotation.tsv'

#                 # Build the full path to the TSV file
#                 tsv_path = os.path.join(annotation_dir, annotation_folder, tsv_file)

#                 if os.path.exists(tsv_path):
#                     # Read the TSV into a dataframe
#                     df_annotation = pd.read_csv(tsv_path, sep='\t', comment='#', header=None, names=['Token_ID', 'Character_Offset', 'Token', 'Annotation_1', 'Annotation_2', 'Annotation_3', 'Annotation_4'], encoding='utf-8', na_filter=False, keep_default_na=False, on_bad_lines='skip')
#                     # Add a column with the name of the txt file
#                     df_annotation['File_Name'] = file

#                     # Append the dataframe to the list
#                     annotation_dfs.append(df_annotation)

#             except Exception as e:
#                 print(f"An error occurred with file {file}. Error: {str(e)}")

# # Concatenate all dataframes
# df_all_source = pd.concat(source_dfs, ignore_index=True)
# df_all_annotation = pd.concat(annotation_dfs, ignore_index=True)

# # Print the combined dataframes to check
# print(df_all_source)
# print(df_all_annotation)

An error occurred with file 019E C E A Armand 1891 Pocket Pharmocopaeia__486_MED.txt. Error: Error tokenizing data. C error: EOF inside string starting at row 1657
An error occurred with file 035E George Eliot 1866 Felix Holt The Radical__67_LIT.txt. Error: Error tokenizing data. C error: EOF inside string starting at row 1460
An error occurred with file 028E 1828 Considerations on the nuisance of coal gas__44_LAW.txt. Error: Error tokenizing data. C error: EOF inside string starting at row 707
An error occurred with file 002E Charles Dickens 1846 Pictures from Italy__52_TRAV.txt. Error: Error tokenizing data. C error: EOF inside string starting at row 3572
An error occurred with file 058E John Charles Sawer 1894 Odorographia__1331_distribution_PER.txt. Error: Error tokenizing data. C error: EOF inside string starting at row 226
An error occurred with file 060E Eugene Rimmel 1865 The Book of Perfumes__964_distribution_PER.txt. Error: Error tokenizing data. C error: EOF inside string st

In [ ]:
df_all_source

Source_Text  \
0   Grooms and Hostlers daily inhale a large quant...   
1   \nOne thing I could not help observing: what b...   
2   \nPART. III.\nAn offer at the Improvement, and...   
3   \nOf the Molucco Jles.\nTHey are numbred five....   
4   Lobsters and Crabs , though exceedingly differ...   
..                                                ...   
80  It is remarkable, however, that though a piece...   
81  The Musk Rat. \n\n\nIn the Memoirs of the Fren...   
82  Isop is reasonable long lasting: young roots a...   
83  What Madness therefore is it, to offer Sacrifi...   
84  It does not seem necessary to enter here into ...   

                                            File_Name  
0   041E Charles Turner Thackrah 1831 The effects ...  
1   033E Daniel Defoe 1722 Journal of a Plague Yea...  
2            039E John Evelyn 1661 Fumifugium_PUB.txt  
3   046E Thomas Herbert 1638 Some years travel__10...  
4   080E Duncan Mcdonald 1800 New town cook__214_d...  
..                                                ...  
80  053E Joseph Priestley 1775 Observations on dif...  
81  057E Charles Lillie 1822 The British Perfumer_...  
82  085E Gervase Markham 1631 A new orchard__20_di...  
83  076E Henry Dodwell 1711 Incense 2__41_distribu...  
84  075E The Case for Incense 1899__38_distributio...  

[85 rows x 2 columns]

In [ ]:
df_all_annotation

Token_ID Character_Offset     Token  Annotation_1 Annotation_2  \
0          1-1              0-6    Grooms  Perceiver[1]            _   
1          1-2             7-10       and  Perceiver[1]            _   
2          1-3            11-19  Hostlers  Perceiver[1]            _   
3          1-4            20-25     daily             _            _   
4          1-5            26-32    inhale   Smell\_Word      *|*|*|*   
...        ...              ...       ...           ...          ...   
95538     9-94        2263-2268     Mercy             _            _   
95539     9-95        2269-2273      Seat             _            _   
95540     9-96        2274-2276        on             _            _   
95541     9-97        2277-2281      High             _            _   
95542     9-98        2281-2282         .             _            _   

                               Annotation_3 Annotation_4  \
0                                         _                
1                                         _                
2                                         _                
3                                         _                
4      1-1[1_0]|1-6[2_0]|1-13[3_0]|2-1[4_0]                
...                                     ...          ...   
95538                                     _                
95539                                     _                
95540                                     _                
95541                                     _                
95542                                     _                

                                               File_Name  
0      041E Charles Turner Thackrah 1831 The effects ...  
1      041E Charles Turner Thackrah 1831 The effects ...  
2      041E Charles Turner Thackrah 1831 The effects ...  
3      041E Charles Turner Thackrah 1831 The effects ...  
4      041E Charles Turner Thackrah 1831 The effects ...  
...                                                  ...  
95538  075E The Case for Incense 1899__38_distributio...  
95539  075E The Case for Incense 1899__38_distributio...  
95540  075E The Case for Incense 1899__38_distributio...  
95541  075E The Case for Incense 1899__38_distributio...  
95542  075E The Case for Incense 1899__38_distributio...  

[95543 rows x 8 columns]

In [ ]:
df_merged = pd.merge(df_all_annotation, df_all_source, on='File_Name', how='outer')

In [ ]:
df_merged.Annotation_1.value_counts()

_                                      114249
                                         3649
Smell\_Word                              1172
Smell\_Source                             631
Quality                                   595
                                        ...  
Smell\_Source[21]|Quality[22]               1
Effect[4]|Smell\_Word[6]                    1
Effect[4]|Quality[5]                        1
Evoked\_Odorant[94]|Smell\_Word[95]         1
Odour\_Carrier[6]|Smell\_Word[7]            1
Name: Annotation_1, Length: 1163, dtype: int64

In [ ]:
# Fill missing values in each row with an empty string
df_merged['Annotation_1'] = df_merged['Annotation_1'].fillna('')
df_merged['Annotation_2'] = df_merged['Annotation_2'].fillna('')
df_merged['Annotation_3'] = df_merged['Annotation_3'].fillna('')
df_merged['Annotation_4'] = df_merged['Annotation_4'].fillna('')

# Merge the columns into a single 'Annotation' column
df_merged['Annotation'] = df_merged['Annotation_1'] + ' ' + df_merged['Annotation_2'] + ' ' + df_merged['Annotation_3'] + ' ' + df_merged['Annotation_4']

# Drop the individual 'Annotation_1' to 'Annotation_4' columns
df_merged = df_merged.drop(columns=['Annotation_1', 'Annotation_2', 'Annotation_3', 'Annotation_4'])

In [ ]:
df_merged

Token_ID Character_Offset       Token  \
0           1-1              0-5       Curry   
1           1-2             6-12      Powder   
2           1-3            13-15          is   
3           1-4            16-26  ordinarily   
4           1-5            27-28           a   
...         ...              ...         ...   
129650    36-17        4370-4378    feverish   
129651    36-18        4379-4387    symptoms   
129652    36-19        4388-4390          as   
129653    36-20        4391-4397      before   
129654    36-21        4397-4398           .   

                                                File_Name  \
0       030E Francis Vacher 1905 Food Inspectors Handb...   
1       030E Francis Vacher 1905 Food Inspectors Handb...   
2       030E Francis Vacher 1905 Food Inspectors Handb...   
3       030E Francis Vacher 1905 Food Inspectors Handb...   
4       030E Francis Vacher 1905 Food Inspectors Handb...   
...                                                   ...   
129650  082E Isabella Beeton 1861 The Book of Househol...   
129651  082E Isabella Beeton 1861 The Book of Househol...   
129652  082E Isabella Beeton 1861 The Book of Househol...   
129653  082E Isabella Beeton 1861 The Book of Househol...   
129654  082E Isabella Beeton 1861 The Book of Househol...   

                                              Source_Text Annotation  
0       Curry Powder is ordinarily a mixture of turmer...     _ _ _   
1       Curry Powder is ordinarily a mixture of turmer...     _ _ _   
2       Curry Powder is ordinarily a mixture of turmer...     _ _ _   
3       Curry Powder is ordinarily a mixture of turmer...     _ _ _   
4       Curry Powder is ordinarily a mixture of turmer...     _ _ _   
...                                                   ...        ...  
129650  It may be distinguished from them by its very ...     _ _ _   
129651  It may be distinguished from them by its very ...     _ _ _   
129652  It may be distinguished from them by its very ...     _ _ _   
129653  It may be distinguished from them by its very ...     _ _ _   
129654  It may be distinguished from them by its very ...     _ _ _   

[129655 rows x 6 columns]

In [ ]:
df_merged

Token_ID Character_Offset       Token  \
0           1-1              0-5       Curry   
1           1-2             6-12      Powder   
2           1-3            13-15          is   
3           1-4            16-26  ordinarily   
4           1-5            27-28           a   
...         ...              ...         ...   
129650    36-17        4370-4378    feverish   
129651    36-18        4379-4387    symptoms   
129652    36-19        4388-4390          as   
129653    36-20        4391-4397      before   
129654    36-21        4397-4398           .   

                                                File_Name  \
0       030E Francis Vacher 1905 Food Inspectors Handb...   
1       030E Francis Vacher 1905 Food Inspectors Handb...   
2       030E Francis Vacher 1905 Food Inspectors Handb...   
3       030E Francis Vacher 1905 Food Inspectors Handb...   
4       030E Francis Vacher 1905 Food Inspectors Handb...   
...                                                   ...   
129650  082E Isabella Beeton 1861 The Book of Househol...   
129651  082E Isabella Beeton 1861 The Book of Househol...   
129652  082E Isabella Beeton 1861 The Book of Househol...   
129653  082E Isabella Beeton 1861 The Book of Househol...   
129654  082E Isabella Beeton 1861 The Book of Househol...   

                                              Source_Text Annotation  
0       Curry Powder is ordinarily a mixture of turmer...     _ _ _   
1       Curry Powder is ordinarily a mixture of turmer...     _ _ _   
2       Curry Powder is ordinarily a mixture of turmer...     _ _ _   
3       Curry Powder is ordinarily a mixture of turmer...     _ _ _   
4       Curry Powder is ordinarily a mixture of turmer...     _ _ _   
...                                                   ...        ...  
129650  It may be distinguished from them by its very ...     _ _ _   
129651  It may be distinguished from them by its very ...     _ _ _   
129652  It may be distinguished from them by its very ...     _ _ _   
129653  It may be distinguished from them by its very ...     _ _ _   
129654  It may be distinguished from them by its very ...     _ _ _   

[129655 rows x 6 columns]

In [ ]:
# remove unwanted characters
df_merged['Annotation'] = df_merged['Annotation'].str.replace(r'\[\d+\]', '')  # remove [numbers]
df_merged['Annotation'] = df_merged['Annotation'].str.replace(r'_', '')  # remove underscores
df_merged['Annotation'] = df_merged['Annotation'].str.replace(r'\\', '')  # remove backslashes

<ipython-input-12-6fd17bbac8a9>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_merged['Annotation'] = df_merged['Annotation'].str.replace(r'\[\d+\]', '')  # remove [numbers]
<ipython-input-12-6fd17bbac8a9>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_merged['Annotation'] = df_merged['Annotation'].str.replace(r'\\', '')  # remove backslashes


In [ ]:
df_merged.Annotation.value_counts()

                                                                        117898
SmellSource                                                               2835
Circumstances                                                             1564
Effect                                                                    1392
Quality                                                                   1187
                                                                         ...  
SmellWord *|* 276-1[1010]|276-8[1020]                                        1
SmellWord * 266-1[1000]                                                      1
SmellWord *|*|*|* 265-28[960]|265-30[970]|265-45[980]|265-47[990]            1
SmellWord *|* 254-40[940]|254-53[950]                                        1
OdourCarrier|SmellWord *|*|*|* 35-4[67]|35-2[57]|35-5[86]|35-14[97]          1
Name: Annotation, Length: 1601, dtype: int64

In [ ]:
# Import regular expressions library
import re

def clean_annotation(annotation):
    # Use a regular expression to find all words in the string.
    # '\b\w+\b' matches word boundaries around one or more word characters.
    words = re.findall(r'\b\w+\b', annotation)

    # Join the words with the pipe character and return the result.
    return '|'.join(words)

# Apply the function to the Annotation column
df_merged['Annotation'] = df_merged['Annotation'].apply(clean_annotation)

In [ ]:
df_merged.Annotation.value_counts()

                                                            117902
SmellSource                                                   2835
Circumstances                                                 1564
Effect                                                        1392
Quality                                                       1187
                                                             ...  
SmellWord|277|2|277|3|277|8|1030|277|10|1040|277|17|1050         1
SmellWord|276|1|1010|276|8|1020                                  1
SmellWord|266|1|1000                                             1
SmellWord|265|28|960|265|30|970|265|45|980|265|47|990            1
OdourCarrier|SmellWord|35|4|67|35|2|57|35|5|86|35|14|97          1
Name: Annotation, Length: 1599, dtype: int64

In [ ]:
# Import regular expressions library
import re

def clean_annotation(annotation):
    # Use a regular expression to find all occurrences of strings with alphabetic characters.
    matches = re.findall(r'\b[A-Za-z_\-]+\b', annotation)

    # Join the matches with the pipe character and return the result.
    return '|'.join(matches)

# Apply the function to the Annotation column
df_merged['Annotation'] = df_merged['Annotation'].apply(clean_annotation)

In [ ]:
df_merged

Token_ID Character_Offset       Token  \
0           1-1              0-5       Curry   
1           1-2             6-12      Powder   
2           1-3            13-15          is   
3           1-4            16-26  ordinarily   
4           1-5            27-28           a   
...         ...              ...         ...   
129650    36-17        4370-4378    feverish   
129651    36-18        4379-4387    symptoms   
129652    36-19        4388-4390          as   
129653    36-20        4391-4397      before   
129654    36-21        4397-4398           .   

                                                File_Name  \
0       030E Francis Vacher 1905 Food Inspectors Handb...   
1       030E Francis Vacher 1905 Food Inspectors Handb...   
2       030E Francis Vacher 1905 Food Inspectors Handb...   
3       030E Francis Vacher 1905 Food Inspectors Handb...   
4       030E Francis Vacher 1905 Food Inspectors Handb...   
...                                                   ...   
129650  082E Isabella Beeton 1861 The Book of Househol...   
129651  082E Isabella Beeton 1861 The Book of Househol...   
129652  082E Isabella Beeton 1861 The Book of Househol...   
129653  082E Isabella Beeton 1861 The Book of Househol...   
129654  082E Isabella Beeton 1861 The Book of Househol...   

                                              Source_Text Annotation  
0       Curry Powder is ordinarily a mixture of turmer...             
1       Curry Powder is ordinarily a mixture of turmer...             
2       Curry Powder is ordinarily a mixture of turmer...             
3       Curry Powder is ordinarily a mixture of turmer...             
4       Curry Powder is ordinarily a mixture of turmer...             
...                                                   ...        ...  
129650  It may be distinguished from them by its very ...             
129651  It may be distinguished from them by its very ...             
129652  It may be distinguished from them by its very ...             
129653  It may be distinguished from them by its very ...             
129654  It may be distinguished from them by its very ...             

[129655 rows x 6 columns]

In [ ]:
df_merged.Annotation.value_counts()

                                                                 117902
SmellSource                                                        2835
Circumstances                                                      1564
Effect                                                             1392
Quality                                                            1187
                                                                  ...  
Time|Perceiver                                                        1
SmellSource|Anaphoric|Anaphoric|Anaphoric|Anaphoric|Anaphoric         1
Effect|Perceiver|Anaphoric                                            1
Time|SmellSource                                                      1
OdourCarrier|SmellWord                                                1
Name: Annotation, Length: 72, dtype: int64

In [ ]:
import re
from collections import defaultdict

def clean_annotation(annotation):
    # Initialize a dictionary to count occurrences of each label
    label_counts = defaultdict(int)

    # Use a regular expression to find all occurrences of labels with alphabetic characters.
    matches = re.findall(r'\b[A-Za-z_\-]+\b', annotation)

    # If there are no matches, return an empty string
    if not matches:
        return ''

    # If label is followed by 'Anaphoric', merge them and treat them as a single label
    merged_matches = []
    for i in range(len(matches)-1):
        if matches[i+1] == 'Anaphoric':
            merged_matches.append(matches[i] + 'Anaphoric')
        elif matches[i] != 'Anaphoric':  # Prevent adding 'Anaphoric' standalone
            merged_matches.append(matches[i])

    # Handle the last match separately to avoid index error
    if matches[-1] != 'Anaphoric':
        merged_matches.append(matches[-1])

    # Count occurrences of each label
    for match in merged_matches:
        label_counts[match] += 1

    # Include each label only once in the cleaned annotation
    cleaned_annotation = '|'.join(label_counts.keys())

    return cleaned_annotation

# Apply the function to the Annotation column
df_merged['Annotation'] = df_merged['Annotation'].apply(clean_annotation)

In [ ]:
df_merged.Annotation.value_counts()

                                   117902
SmellSource                          2835
Circumstances                        1564
Effect                               1394
Quality                              1187
                                    ...  
Time|SmellSource                        1
Quality|SmellSource                     1
Perceiver|OdourCarrierAnaphoric         1
Time|Perceiver                          1
OdourCarrier|SmellWord                  1
Name: Annotation, Length: 64, dtype: int64

In [ ]:
df_merged['Unique_Label_Combinations'] = df_merged['Annotation'].apply(lambda x: '-'.join(sorted(x.split('|'))))

In [ ]:
df_merged['Label_ID'] = df_merged['Unique_Label_Combinations'].astype('category').cat.codes

In [ ]:
df_merged

Token_ID Character_Offset       Token  \
0           1-1              0-5       Curry   
1           1-2             6-12      Powder   
2           1-3            13-15          is   
3           1-4            16-26  ordinarily   
4           1-5            27-28           a   
...         ...              ...         ...   
129650    36-17        4370-4378    feverish   
129651    36-18        4379-4387    symptoms   
129652    36-19        4388-4390          as   
129653    36-20        4391-4397      before   
129654    36-21        4397-4398           .   

                                                File_Name  \
0       030E Francis Vacher 1905 Food Inspectors Handb...   
1       030E Francis Vacher 1905 Food Inspectors Handb...   
2       030E Francis Vacher 1905 Food Inspectors Handb...   
3       030E Francis Vacher 1905 Food Inspectors Handb...   
4       030E Francis Vacher 1905 Food Inspectors Handb...   
...                                                   ...   
129650  082E Isabella Beeton 1861 The Book of Househol...   
129651  082E Isabella Beeton 1861 The Book of Househol...   
129652  082E Isabella Beeton 1861 The Book of Househol...   
129653  082E Isabella Beeton 1861 The Book of Househol...   
129654  082E Isabella Beeton 1861 The Book of Househol...   

                                              Source_Text Annotation  \
0       Curry Powder is ordinarily a mixture of turmer...              
1       Curry Powder is ordinarily a mixture of turmer...              
2       Curry Powder is ordinarily a mixture of turmer...              
3       Curry Powder is ordinarily a mixture of turmer...              
4       Curry Powder is ordinarily a mixture of turmer...              
...                                                   ...        ...   
129650  It may be distinguished from them by its very ...              
129651  It may be distinguished from them by its very ...              
129652  It may be distinguished from them by its very ...              
129653  It may be distinguished from them by its very ...              
129654  It may be distinguished from them by its very ...              

       Unique_Label_Combinations  Label_ID  
0                                        0  
1                                        0  
2                                        0  
3                                        0  
4                                        0  
...                          ...       ...  
129650                                   0  
129651                                   0  
129652                                   0  
129653                                   0  
129654                                   0  

[129655 rows x 8 columns]

In [ ]:
df_merged.to_csv('/content/df_merged')

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!cp '/content/df_merged' "drive/My Drive/"

In [ ]:
df_merged

Token_ID Character_Offset       Token  \
0           1-1              0-5       Curry   
1           1-2             6-12      Powder   
2           1-3            13-15          is   
3           1-4            16-26  ordinarily   
4           1-5            27-28           a   
...         ...              ...         ...   
129650    36-17        4370-4378    feverish   
129651    36-18        4379-4387    symptoms   
129652    36-19        4388-4390          as   
129653    36-20        4391-4397      before   
129654    36-21        4397-4398           .   

                                                File_Name  \
0       030E Francis Vacher 1905 Food Inspectors Handb...   
1       030E Francis Vacher 1905 Food Inspectors Handb...   
2       030E Francis Vacher 1905 Food Inspectors Handb...   
3       030E Francis Vacher 1905 Food Inspectors Handb...   
4       030E Francis Vacher 1905 Food Inspectors Handb...   
...                                                   ...   
129650  082E Isabella Beeton 1861 The Book of Househol...   
129651  082E Isabella Beeton 1861 The Book of Househol...   
129652  082E Isabella Beeton 1861 The Book of Househol...   
129653  082E Isabella Beeton 1861 The Book of Househol...   
129654  082E Isabella Beeton 1861 The Book of Househol...   

                                              Source_Text Annotation  \
0       Curry Powder is ordinarily a mixture of turmer...              
1       Curry Powder is ordinarily a mixture of turmer...              
2       Curry Powder is ordinarily a mixture of turmer...              
3       Curry Powder is ordinarily a mixture of turmer...              
4       Curry Powder is ordinarily a mixture of turmer...              
...                                                   ...        ...   
129650  It may be distinguished from them by its very ...              
129651  It may be distinguished from them by its very ...              
129652  It may be distinguished from them by its very ...              
129653  It may be distinguished from them by its very ...              
129654  It may be distinguished from them by its very ...              

       Unique_Label_Combinations  Label_ID  
0                                        0  
1                                        0  
2                                        0  
3                                        0  
4                                        0  
...                          ...       ...  
129650                                   0  
129651                                   0  
129652                                   0  
129653                                   0  
129654                                   0  

[129655 rows x 8 columns]

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.6 MB/s eta 0:00:00


In [ ]:
df_merged

Token_ID Character_Offset       Token  \
0           1-1              0-5       Curry   
1           1-2             6-12      Powder   
2           1-3            13-15          is   
3           1-4            16-26  ordinarily   
4           1-5            27-28           a   
...         ...              ...         ...   
129650    36-17        4370-4378    feverish   
129651    36-18        4379-4387    symptoms   
129652    36-19        4388-4390          as   
129653    36-20        4391-4397      before   
129654    36-21        4397-4398           .   

                                                File_Name  \
0       030E Francis Vacher 1905 Food Inspectors Handb...   
1       030E Francis Vacher 1905 Food Inspectors Handb...   
2       030E Francis Vacher 1905 Food Inspectors Handb...   
3       030E Francis Vacher 1905 Food Inspectors Handb...   
4       030E Francis Vacher 1905 Food Inspectors Handb...   
...                                                   ...   
129650  082E Isabella Beeton 1861 The Book of Househol...   
129651  082E Isabella Beeton 1861 The Book of Househol...   
129652  082E Isabella Beeton 1861 The Book of Househol...   
129653  082E Isabella Beeton 1861 The Book of Househol...   
129654  082E Isabella Beeton 1861 The Book of Househol...   

                                              Source_Text Annotation  \
0       Curry Powder is ordinarily a mixture of turmer...              
1       Curry Powder is ordinarily a mixture of turmer...              
2       Curry Powder is ordinarily a mixture of turmer...              
3       Curry Powder is ordinarily a mixture of turmer...              
4       Curry Powder is ordinarily a mixture of turmer...              
...                                                   ...        ...   
129650  It may be distinguished from them by its very ...              
129651  It may be distinguished from them by its very ...              
129652  It may be distinguished from them by its very ...              
129653  It may be distinguished from them by its very ...              
129654  It may be distinguished from them by its very ...              

       Unique_Label_Combinations  Label_ID  
0                                        0  
1                                        0  
2                                        0  
3                                        0  
4                                        0  
...                          ...       ...  
129650                                   0  
129651                                   0  
129652                                   0  
129653                                   0  
129654                                   0  

[129655 rows x 8 columns]

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the output file path in Google Drive
output_file_path = '/content/drive/MyDrive/df_merged.csv'

# Save the dataframe as a CSV file
df_merged.to_csv(output_file_path, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the output file path in Google Drive
output_file_path = '/content/drive/MyDrive/df_merged.parquet'

# Save the dataframe as a Parquet file
df_merged.to_parquet(output_file_path, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!du -h /content/drive/MyDrive/df_merged.zip

271M	/content/drive/MyDrive/df_merged.zip


In [ ]:
!zip /content/drive/MyDrive/df_merged.zip /content/drive/MyDrive/df_merged

  adding: content/drive/MyDrive/df_merged (deflated 88%)


In [ ]:
# Define the path to the CSV file in Google Drive
file_path = '/content/drive/MyDrive/df_merged.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

In [ ]:
df

Token_ID Character_Offset       Token  \
0           1-1              0-5       Curry   
1           1-2             6-12      Powder   
2           1-3            13-15          is   
3           1-4            16-26  ordinarily   
4           1-5            27-28           a   
...         ...              ...         ...   
129650    36-17        4370-4378    feverish   
129651    36-18        4379-4387    symptoms   
129652    36-19        4388-4390          as   
129653    36-20        4391-4397      before   
129654    36-21        4397-4398           .   

                                                File_Name  \
0       030E Francis Vacher 1905 Food Inspectors Handb...   
1       030E Francis Vacher 1905 Food Inspectors Handb...   
2       030E Francis Vacher 1905 Food Inspectors Handb...   
3       030E Francis Vacher 1905 Food Inspectors Handb...   
4       030E Francis Vacher 1905 Food Inspectors Handb...   
...                                                   ...   
129650  082E Isabella Beeton 1861 The Book of Househol...   
129651  082E Isabella Beeton 1861 The Book of Househol...   
129652  082E Isabella Beeton 1861 The Book of Househol...   
129653  082E Isabella Beeton 1861 The Book of Househol...   
129654  082E Isabella Beeton 1861 The Book of Househol...   

                                              Source_Text Annotation  \
0       Curry Powder is ordinarily a mixture of turmer...        NaN   
1       Curry Powder is ordinarily a mixture of turmer...        NaN   
2       Curry Powder is ordinarily a mixture of turmer...        NaN   
3       Curry Powder is ordinarily a mixture of turmer...        NaN   
4       Curry Powder is ordinarily a mixture of turmer...        NaN   
...                                                   ...        ...   
129650  It may be distinguished from them by its very ...        NaN   
129651  It may be distinguished from them by its very ...        NaN   
129652  It may be distinguished from them by its very ...        NaN   
129653  It may be distinguished from them by its very ...        NaN   
129654  It may be distinguished from them by its very ...        NaN   

       Unique_Label_Combinations  Label_ID  
0                            NaN         0  
1                            NaN         0  
2                            NaN         0  
3                            NaN         0  
4                            NaN         0  
...                          ...       ...  
129650                       NaN         0  
129651                       NaN         0  
129652                       NaN         0  
129653                       NaN         0  
129654                       NaN         0  

[129655 rows x 8 columns]